# Hero Match-Ups

Use logistic regression to determine which heroes counter other heroes and which have synergy with other heroes.

In [1]:
import sqlite3
import meta
import os
import ujson as json
import pandas as pd
import numpy as np
import hero_analysis
from sklearn.linear_model import LogisticRegression

HERO='juggernaut'

In [2]:
DB_FILES = ['matches_1_all_2020010710.db']

In [3]:
rows=[]
for db_file in DB_FILES:
    conn = sqlite3.connect(db_file)
    c = conn.cursor()
    c.execute("SELECT match_id, radiant_heroes, dire_heroes, items, gold_spent, radiant_win FROM {}".format(
        os.environ['DOTA_SQL_STATS_TABLE']))
    rows.extend(c.fetchall())
    conn.close()
    
print("{0} matches found in database".format(len(rows)))

131661 matches found in database


In [4]:
enemy_list = []
ally_list = []
for match_id, radiant_heroes, dire_heroes, items, gold_spent, radiant_win in rows:
    a, b = hero_analysis.unpack_match(match_id, HERO, radiant_heroes, dire_heroes, radiant_win)
    if not(a==[]):
        enemy_list.extend(a)
    if not(b==[]):
        ally_list.extend(b)

# List of Enemy Counters

In [ ]:
df_enemy=pd.DataFrame(enemy_list)
df_enemy.columns=["match_id", "target_hero", "radiant_flag", "enemy_hero", "win"]
df_enemy.to_csv("temp_enemy_flat.csv")

In [ ]:
df_enemy['p1']=1
df_enemy=df_enemy.pivot_table(index=['match_id', 'target_hero', 'radiant_flag', 'win'], columns='enemy_hero', values='p1').fillna(0)
df_enemy.reset_index(inplace=True)

In [ ]:
hero_columns=[t for t in df_enemy.columns if t not in ['match_id', 'target_hero', 'radiant_flag', 'win']]

In [ ]:
X1=np.array([int(t) for t in df_enemy['radiant_flag']]).reshape(-1,1)
X2=df_enemy[hero_columns].values
y=[int(t) for t in df_enemy['win']]

In [ ]:
X=np.concatenate([X1, X2], axis=1)
X_labels=["radiant"]+hero_columns

In [ ]:
lr=LogisticRegression()
_ = lr.fit(X,y)

In [ ]:
results=pd.DataFrame({
    'variable' : X_labels,
    'coeff' : lr.coef_[0]
})
results.to_csv("temp_results.csv")

# List of Synergistic Teammates

In [5]:
df_ally=pd.DataFrame(ally_list)
df_ally.columns=["match_id", "target_hero", "radiant_flag", "ally_hero", "win"]
df_ally.to_csv("temp_ally_flat.csv")

In [6]:
df_ally['p1']=1
df_ally=df_ally.pivot_table(index=['match_id', 'target_hero', 'radiant_flag', 'win'], columns='ally_hero', values='p1').fillna(0)
df_ally.reset_index(inplace=True)

In [11]:
hero_columns=[t for t in df_ally.columns if t not in ['match_id', 'target_hero', 'radiant_flag', 'win']]

In [13]:
X1=np.array([int(t) for t in df_ally['radiant_flag']]).reshape(-1,1)
X2=df_ally[hero_columns].values
y=[int(t) for t in df_ally['win']]

In [15]:
X=np.concatenate([X1, X2], axis=1)
X_labels=["radiant"]+hero_columns
lr=LogisticRegression()
_ = lr.fit(X,y)

results=pd.DataFrame({
    'variable' : X_labels,
    'coeff' : lr.coef_[0]
})
results.to_csv("temp_ally_results.csv")